### <center> Projet: Utilisation de Beautiful Soup pour extraire les données </center>

À l'issue de ce TP, vous serez en mesure de développer un programme capable de récupérer les données des offres d'emplois de data scientist.

Etapes : 
- Vérifier si le site autorise le scraping
- Obtenir la page web
- Extraire les données du site
- Gérer les erreurs
- Stocker les données extraites




### Analyse du robots.txt

In [1]:
import requests
from urllib.parse import urljoin

def get_robots_txt(url):
        """Récupère le contenu du fichier robots.txt"""
        try:
            name = url.split(".")[0]
            # Construire l'URL du robots.txt
            if not url.startswith(('http://', 'https://')):
                url = 'https://' + url
            robots_url = urljoin(url, '/robots.txt')
            
            # Faire la requête
            response = requests.get(robots_url)
            
            if response.status_code == 200:
                with open(f"robot_{name}.txt", "w") as file:
                    file.write(response.text)
                print("fichier crée")
            else:
                print(f"Erreur {response.status_code} pour {robots_url}")
               
        except Exception as e:
            print(f"Erreur pget_robots_txtour {url}: {str(e)}")


In [3]:
get_robots_txt("linkedin.com")

fichier crée


In [5]:
get_robots_txt("welcometothejungle.com")

fichier crée


### Obtenir la page


À ce stade, l'étudiant devra effectuer deux tests. Premier test sans spécifier la variable `headers` dans la `request.get`. Second test en incluant la variable `headers`.

Quelle interprétation peut-on en tirer ?

In [7]:
root = "https://www.analyticsinsight.net/data-science/top-50-data-science-jobs-on-linkedin"

In [25]:
import requests
from bs4 import BeautifulSoup

def get_soup(url):
        """Obtenir le soup d'une page avec gestion des erreurs"""
        headers = {
            'User-Agent': 'Mozilla1/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        try:
            response = requests.get(url,headers=headers)
            if response.status_code == 200:
                return BeautifulSoup(response.content, 'html.parser')
            else:
                print(f"Erreur {response.status_code}")
                return None
        except Exception as e:
            print(f"Erreur: {e}")
            return None

In [13]:
soup2 = get_soup(root)

In [49]:
#soup2

## TOP 50 DES JOBS DATA SCIENCE

### Extraire les données de la page

différence entre find() and select() [stackoverflow](https://stackoverflow.com/questions/38028384/beautifulsoup-difference-between-find-and-select)

#### Trouver l'identifiant du conteneur des produits 

In [15]:
job_info2 = soup2.select("div[data-test-id='text']")

In [17]:
len(job_info2)

103

In [19]:
job_info2[100]

<div class="arrow-component arr--text-element text-m_textElement__e3QEt text-m_dark__1TC18" data-test-id="text" style="color:#000"><p><strong>Release Date:</strong> 1st July 2024</p><p><strong>City:</strong> Hyderabad</p><p><strong>Required Experience:</strong> 5+ years</p><p><strong>Salary Range:</strong> Not Disclosed</p><p><strong>Employment Type:</strong> Full Time</p><p><strong>Industry:</strong> Digital Transformation</p><p><strong>Key Skills:</strong> Designing, Building, and Deploying ML and DL models, PyTorch, TensorFlow</p><p><strong>Source:</strong> LinkedIn</p><p><strong>Reference Link:</strong> <a aria-label="content" href="https://www.linkedin.com/posts/dr-sagar-a-k-94951565_digital-transformation-solutions-services-activity-7213442242621947905-s1nA?utm_source=share&amp;utm_medium=member_desktop" target="_blank">Web Synergies Machine Learning Engineer Job</a></p></div>

In [21]:
job_info2[2].select_one("p:contains('Release Date')").get_text(strip=True).replace("Release Date: ", "")

C:\Users\celia\anaconda3\Lib\site-packages\soupsieve\css_parser.py:856: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(  # noqa: B028


'Release Date:30th June 2024'

In [23]:
job_info2[2].select_one("p:contains('City')").get_text(strip=True).split("City: ")[0]

'City:Remote'

In [25]:
job_info2[2].select_one("p:contains('Required Experience')").get_text(strip=True).split("Required Experience: ")[0]

'Required Experience:0-2 years'

In [27]:
job_info2[2].select_one("p:contains('Salary Range')").get_text(strip=True).split("Salary Range: ")[0]

'Salary Range:Not Disclosed'

In [29]:
job_info2[2].select_one("p:contains('Employment Type')").get_text(strip=True).split("Employment Type: ")[0]

'Employment Type:Full Time'

In [31]:
job_info2[2].select_one("p:contains('Industry')").get_text(strip=True).split("Industry: ")[0]

'Industry:Food Delivery Platform'

In [33]:
job_info2[2].select_one("p:contains('Key Skills')").get_text(strip=True).split("Key Skills: ")[0]

'Key Skills:Python, SQL, Spark, TensorFlow'

#### Prendre une offre d'emploi et extraire tous les éléments

Questions à répondre : 
- Quelle est le critère sélection (id ou class) ?
- Cette approche fonctionne-t-elle avec tous les offre ?

In [35]:
Date = []
Salaires_moyens = []
Compétences_demandées= []
Experience = []
Secteur_Activite = []
Localisation= []
Types_contrats= []

#### Scrapping de toutes les 50 offres 

In [37]:
i = 2
while i < (len(job_info2) - 2): 
    try:
        release_date_tag = job_info2[i].select_one("p:contains('Release Date')")
        if release_date_tag:
            Date.append(release_date_tag.get_text(strip=True).replace("Release Date: ", ""))
        else:
            Date.append(None)
    except Exception as e:
        Date.append(None)
    
    try:
        city_tag = job_info2[i].select_one("p:contains('City')")
        if city_tag:
            Localisation.append(city_tag.get_text(strip=True).replace("City: ", ""))
        else:
            Localisation.append(None)
    except Exception as e:
        Localisation.append(None)
    
    try:
        experience_tag = job_info2[i].select_one("p:contains('Required Experience')")
        if experience_tag:
            Experience.append(experience_tag.get_text(strip=True).replace("Required Experience: ", ""))
        else:
            Experience.append(None)
    except Exception as e:
        Experience.append(None)
    
    try:
        skills_tag = job_info2[i].select_one("p:contains('Key Skills')")
        if skills_tag:
            Compétences_demandées.append(skills_tag.get_text(strip=True).replace("Key Skills: ", ""))
        else:
            Compétences_demandées.append(None)
    except Exception as e:
        Compétences_demandées.append(None)
    
    try:
        salary_tag = job_info2[i].select_one("p:contains('Salary Range')")
        if salary_tag:
            Salaires_moyens.append(salary_tag.get_text(strip=True).replace("Salary Range: ", ""))
        else:
            Salaires_moyens.append(None)
    except Exception as e:
        Salaires_moyens.append(None)
    
    try:
        contract_tag = job_info2[i].select_one("p:contains('Employment Type')")
        if contract_tag:
            Types_contrats.append(contract_tag.get_text(strip=True).replace("Employment Type: ", ""))
        else:
            Types_contrats.append(None)
    except Exception as e:
        Types_contrats.append(None)
    
    try:
        industry_tag = job_info2[i].select_one("p:contains('Industry')")
        if industry_tag:
            Secteur_Activite.append(industry_tag.get_text(strip=True).replace("Industry: ", ""))
        else:
            Secteur_Activite.append(None)
    except Exception as e:
        Secteur_Activite.append(None)

    i = i + 2



    

In [39]:
import pandas as pd

In [41]:
data2 = {
    'Date': Date,
    'Localisation': Localisation,
    'Experience': Experience,
    'Compétences_demandées': Compétences_demandées,
    'Salaires_moyens': Salaires_moyens,
    'Types_contrats': Types_contrats,
    'Secteur_Activite': Secteur_Activite
}

df2 = pd.DataFrame(data2)
#print(df2)

In [43]:
df2.to_csv(f'Offre_DS_Lk_50.csv', index=False, encoding='utf-8')


## WELCOME TO THE JUNGLE

#### Trouver l'identifiant du conteneur des produits

##### Approche iterative

In [67]:
soup = get_soup("https://www.welcometothejungle.com/fr/companies/zeffy/jobs/full-stack-data-scientist_paris?q=8f542bd7644f22ec43784937c077daee&o=8be82671-b057-4238-9091-df5a1069e25a")

In [14]:
#soup

In [7]:
job_infos

<div class="sc-bXCLTC cWuusC"><div class="sc-bXCLTC hGtksh"><div class="sc-bXCLTC hdepoj"><div class="sc-kgOKUu jtkJmT" variant="default" w="fit-content"><i class="sc-camqpD ixTeso wui-icon-font" name="contract"></i>CDI</div><div class="sc-kgOKUu jtkJmT" variant="default" w="fit-content"><i class="sc-camqpD cdHLAr wui-icon-font" name="location"></i><span class="sc-cIfMiO eeuvdh"><span class="sc-EElJA lfdfiP">Paris</span></span></div><div class="sc-kgOKUu jtkJmT" variant="default" w="fit-content"><i class="sc-camqpD eTbujJ wui-icon-font" name="salary"></i><span class="sc-bXCLTC kJcLKT">Salaire : </span>50K à 70K €</div><div class="sc-kgOKUu jtkJmT" variant="default" w="fit-content"><i class="sc-camqpD gOcjnm wui-icon-font" name="remote"></i><span>Télétravail fréquent</span><span class="sc-bXCLTC iGeUCm"><i class="sc-camqpD joAPZn wui-icon-font" name="information_outline"></i></span><span hidden="" id=":Rbdj2an3a:" style="position:fixed"></span></div><div class="sc-kgOKUu jtkJmT" variant

In [9]:
job_infos.text.strip()

'CDIParisSalaire : 50K à 70K\xa0€Télétravail fréquentExpérience : > 6 mois'

In [22]:
elements = soup.find_all('div', class_="sc-kgOKUu jtkJmT")

In [30]:
for element in elements:
        print(element.text.strip())

CDI
Paris
Salaire : 50K à 70K €
Télétravail fréquent
Expérience : > 6 mois


In [32]:
competences_elements = soup.find_all('div', class_="sc-kgOKUu hRRfVz")

In [36]:
for element in competences_elements:
    print(element.text.strip())


In [38]:
additional_info = soup.find('div', class_="sc-bXCLTC dBpdut")

In [40]:
additional_info.text.strip()

'ZeffyFinTech / InsurTech, SocialTech / GreenTech35 collaborateursCréée en 2017Âge moyen : 29 ans42%58%Voir le siteVoir toutes les offres5Qui sont-ils ?Chaque année, les associations et organismes de charité dépensent 3 milliards de dollar de dons en frais de transaction. Zeffy met un terme à cette perte, en offrant la première plateforme de collecte de fonds 100% gratuite.EngagementsB CorpLe lieu de travail14 Cité Griset, 75011 Paris, France'

##### Généralisation 

In [77]:
def Job_Info(url):
    soup = get_soup(url)
    
    Liste = []

    job_infos = soup.find('div', class_="sc-bXCLTC cWuusC")
    
    if job_infos and job_infos.text.strip():
        Liste.append(job_infos.text.strip())
    else:
        Liste.append(None)

    Coompetences = []
    competences_elements = soup.find_all('div', class_="sc-kgOKUu hRRfVz")
    for element in competences_elements:
        if element:
            Coompetences.append(element.text.strip())
        else:
            Coompetences.append(None)
    Liste.append(Coompetences)

    additional_info = soup.find('div', class_="sc-bXCLTC dBpdut")
    if additional_info:
        Liste.append(additional_info.text.strip())
    else:
        Liste.append(None)
    
    return Liste


#### Approche en ce qui concerne le lien 

In [153]:
root="https://www.welcometothejungle.com/fr/jobs?query=data%20scientist&aroundQuery=worldwide&page=1"

In [155]:
soup4 = get_soup(root)
    

In [127]:
#soup4

In [157]:
Lien_T = soup4.find_all('div',class_='sc-bXCLTC kkKAOM ais-Hits-list-item')

In [1]:
#Lien_T

#### Combinaison entre Selenium et Beautiful soup

In [5]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


In [3]:
def get_elements_by_class(url, loading_time=30, target_class="sc-bXCLTC kkKAOM ais-Hits-list-item"):
    
    Liens1 =[]

    # Initialisez le driver Chrome
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    try:
        # Chargez l'URL
        driver.get(url)

        # Attendez que le contenu de la page soit chargé pendant le temps défini
        time.sleep(loading_time)  # Utilisez `sleep` pour attendre la durée spécifiée

        # Obtenez le contenu de la page
        page_source = driver.page_source

    except Exception as e:
        print(f"Erreur: {e}")
        return None

    finally:
         # Fermez le driver
        driver.quit()

    # Utilisez BeautifulSoup pour analyser le contenu de la page
    soup = BeautifulSoup(page_source, 'html.parser')

    # Trouvez les éléments ayant la classe spécifiée
    elements = soup.find_all(class_=target_class)

    

    for j in range(len(elements)) :
        
        Liens1.append(elements[j].select_one('a')['href'])

    return Liens1


##### Généralisation

In [13]:
Liens= []
for i in range(1, 13):
    
    url = f"https://www.welcometothejungle.com/fr/jobs?query=data%20scientist&aroundQuery=worldwide&page=" + str(i)

    Lien_T = get_elements_by_class(url, loading_time=30, target_class="sc-bXCLTC kkKAOM ais-Hits-list-item")
    
    for j in range(len(Lien_T)) :
        
        Liens.append(Lien_T[j])
    




In [25]:
#Liens

In [15]:
len(Liens)

350

In [19]:
#Liens

#### Base de donnée de liens

In [21]:
import pandas as pd

data_liens = {'Liens': Liens}
df_liens = pd.DataFrame(data_liens)

In [23]:
df_liens.to_csv(f'Base_Liens.csv', index=False, encoding='utf-8')

#### Base de donnée des offres de travail 

In [29]:
Infos_jobs = []
Compétences_demandées = []
Secteur_Activite = []


In [31]:
base ="https://www.welcometothejungle.com"
data = {
    'Infos_jobs': Infos_jobs,
    'Compétences_demandées': Compétences_demandées,
    'Secteur_Activite': Secteur_Activite
}


In [33]:
from tqdm import tqdm

#### Scraping des données

In [35]:
for r in tqdm(Liens) :
    url = base+r
    
    Job_Info(url)
    
    Infos_jobs.append(Job_Info(url)[0])
    Compétences_demandées.append(Job_Info(url)[1])
    Secteur_Activite.append(Job_Info(url)[2])
    

100%|████████████████████████████████████████████████████████████████████████████████| 350/350 [22:16<00:00,  3.82s/it]


In [39]:
#data

In [41]:
df = pd.DataFrame(data)

In [43]:
df

,Infos_jobs,Compétences_demandées,Secteur_Activite
0,Résumé du posteCDISuresnesSalaire : Non spécif...,"[Leadership, Compétences en communication, Pys...","AXABanque, Assurance, FinTech / InsurTech21889..."
1,Résumé du posteCDINantesSalaire : Non spécifié...,"[Gitlab, Sonarqube, Sql, Python, Klaxoon]",BNP ParibasBanque183883 collaborateursCréée en...
2,CDIBoulogne-BillancourtSalaire : Non spécifiéT...,[],Renault DigitalIntelligence artificielle / Mac...
3,Résumé du posteStageSophia-AntipolisSalaire : ...,"[Créativité, Java, Scala, Segment, Sonar]","ThalesLogiciels, Cybersécurité, Aéronautique /..."
4,Alternance(12 mois)Charenton-le-PontSalaire : ...,[],"Groupe BPCEBanque, FinTech / InsurTech, Financ..."
...,...,...,...
345,Résumé du posteCDIParisSalaire : Non spécifiéD...,"[Compétences en communication, Compétences en ...","MetroscopeLogiciels, Intelligence artificielle..."
346,Résumé du posteCDINiortSalaire : Non spécifiéT...,"[Segment, Aws]",MicropoleIT / Digital1200 collaborateursCréée ...
347,StagePierrelatteSalaire : Non spécifiéTélétrav...,[],"OranoIngénieries Spécialisées, Energie17500 co..."
348,Résumé du posteCDILevallois-PerretSalaire : No...,"[Azure, Segment, Aws]",MicropoleIT / Digital1200 collaborateursCréée ...


In [47]:
df.to_csv(f'Base_Final.csv', index=False, encoding='utf-8')
